### Utilities

In [48]:
from __future__ import absolute_import
from __future__ import division
import tensorflow as tf
import pandas as pd
import numpy as np
import scipy.sparse as sp
import os as os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from time import time

# Batch handling
def remove_item(feature_mask, users, item):
    flag = 0
    for i in range(len(users)):
        if users[i] == item:
            users[i] = users[-1]
            users[-1] = feature_mask
            flag = 1
            break
    return len(users) - flag

def add_mask(feature_mask, features, num_max):
    #uniformalize the length of each batch
    for i in range(len(features)):
        features[i] = features[i] + [feature_mask] * (num_max+1 - len(features[i]))
    return features


def shuffle(trainMatrix, trainList, _num_negatives):

    # Get train data user
    _user_input, _item_input, _labels, _batch_length = [],[],[],[]
    train = trainMatrix
    trainList = trainList
    
    _num_items = train.shape[1]
    _num_users = train.shape[0]
    
    print(_num_users)
    print(_num_items)
    
    for u in range(_num_users):
        if u == 0:
            _batch_length.append((1+_num_negatives) * len(trainList[u]))
        else:
            _batch_length.append((1+_num_negatives) * len(trainList[u])+_batch_length[u-1])
        for i in trainList[u]:
            # positive instance
            _user_input.append(u)
            _item_input.append(i)
            _labels.append(1)
            # negative instances
            for t in range(_num_negatives):
                j = np.random.randint(_num_items)
                while j in trainList[u]:
                    j = np.random.randint(_num_items)
                _user_input.append(u)
                _item_input.append(j)
                _labels.append(0)

    _num_batch = len(_batch_length)
     
    # Preprocess
    user_input_list, num_idx_list, item_input_list, labels_list = [], [], [], []
    for i in range(int(_num_batch)):
        
        # Train Batch User
        user_list1, num_list1, item_list1, labels_list1 = [],[],[],[]
        if i == 0:
            begin = 0
        else:
            begin = _batch_length[i-1]
        batch_index = list(range(begin, _batch_length[i]))
        np.random.shuffle(batch_index)
        
        for idx in batch_index:
            user_idx = _user_input[idx]
            item_idx = _item_input[idx]
            nonzero_row = []
            nonzero_row += trainList[user_idx]
            num_list1.append(remove_item(_num_items, nonzero_row, item_idx))
            user_list1.append(nonzero_row)
            item_list1.append(item_idx)
            labels_list1.append(_labels[idx])
        user_input = np.array(add_mask(_num_items, user_list1, max(num_list1)))
        num_idx = np.array(num_list1)
        item_input = np.array(item_list1)
        labels = np.array(labels_list1)
        
        ui, ni, ii, l = user_input, num_idx, item_input, labels
     
    
        user_input_list.append(ui)
        num_idx_list.append(ni)
        item_input_list.append(ii)
        labels_list.append(l)


    return  user_input_list, num_idx_list, item_input_list, labels_list
        




# Dataset

def load_rating_file_as_list(filename):
    ratingList = []
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item = int(arr[0]), int(arr[1])
            ratingList.append([user, item])
            line = f.readline()
    return ratingList

def load_negative_file(filename):
    negativeList = []
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            negatives = []
            for x in arr[1: ]:
                negatives.append(int(x))
            negativeList.append(negatives)
            line = f.readline()
    return negativeList

def load_training_file_as_matrix(filename):
    # Get number of users and items
    num_users, num_items = 0, 0
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            u, i = int(arr[0]), int(arr[1])
            num_users = max(num_users, u)
            num_items = max(num_items, i)
            line = f.readline()
    # Construct matrix
    mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
            if (rating > 0):
                mat[user, item] = 1.0
            line = f.readline()
    print("already load the trainMatrix...")
    return mat


def load_training_file_as_list(filename):
    # Get number of users and items
    u_ = 0
    lists, items = [], []
    with open(filename, "r") as f:
        line = f.readline()
        index = 0
        while line != None and line != "":
            arr = line.split("\t")
            u, i = int(arr[0]), int(arr[1])
            if u_ < u:
                index = 0
                lists.append(items)
                items = []
                u_ += 1
            index += 1
            if index < 300:
                items.append(i)
            line = f.readline()
    lists.append(items)
    print("already load the trainList...")
    return lists


# def batch_norm_layer(x, train_phase, scope_bn):
#     bn_train = batch_norm(x, decay=0.9, center=True, scale=True, updates_collections=None,
#                           is_training=True, reuse=None, trainable=True, scope=scope_bn)
#     bn_inference = batch_norm(x, decay=0.9, center=True, scale=True, updates_collections=None,
#                               is_training=False, reuse=True, trainable=True, scope=scope_bn)
#     z = tf.cond(train_phase, lambda: bn_train, lambda: bn_inference)
#     return z


def training_batch(batch_index, model, sess, batches):
    for index in batch_index:
        user_input, num_idx, item_input, labels = data.batch_gen(batches, index)
        feed_dict = {model.user_input: user_input, model.num_idx: num_idx[:, None],
                     model.item_input: item_input[:, None],
                     model.labels: labels[:, None], model.is_train_phase: True}
        sess.run(model.optimizer, feed_dict)


def training_loss(model, sess, batches):
    train_loss = 0.0
    num_batch = len(batches[1])
    for index in range(num_batch):
        user_input, num_idx, item_input, labels = data.batch_gen(batches, index)
        feed_dict = {model.user_input: user_input, model.num_idx: num_idx[:, None],
                     model.item_input: item_input[:, None],
                     model.labels: labels[:, None], model.is_train_phase: True}
        train_loss += sess.run(model.loss, feed_dict)
    return train_loss / num_batch
    
    


### Dataset Preparation

In [31]:
path = "../datasets/Data/ml-1m"

trainMatrix = load_training_file_as_matrix(path + ".train.rating")
trainList = load_training_file_as_list(path + ".train.rating")
testRatings = load_rating_file_as_list(path + ".test.rating")
testNegatives = load_negative_file(path + ".test.negative")
assert len(testRatings) == len(testNegatives)
num_users, num_items = trainMatrix.shape


already load the trainMatrix...
already load the trainList...


In [43]:
epochs = 100
learning_rate = 0.01
embedding_size = 16
alpha = 0.5
verbose = 1
n_hidden = [64, 32, 16]
regs = [1e-06, 1e-06]
train_loss = 1
reg_W = [0.1,0.1,0.1,0.1]
batch_choice = "user"
use_batch_norm = False
num_negatives = 2
lambda_bilinear = 1 #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
gamma_bilinear = 1 #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


### Building the Model


In [33]:
# Creating Placeholders
user_input = tf.placeholder(tf.int32, shape=[None, None])  # the index of users
num_idx = tf.placeholder(tf.float32, shape=[None, 1])  # the number of items rated by users
item_input = tf.placeholder(tf.int32, shape=[None, 1])  # the index of items
labels = tf.placeholder(tf.float32, shape=[None, 1])  # the ground truth
is_train_phase = tf.placeholder(tf.bool)


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# Creating Variables
c1 = tf.Variable(tf.truncated_normal(shape=[num_items, embedding_size], 
                                     mean=0.0, stddev=0.01), name='c1', dtype=tf.float32)
c2 = tf.constant(0.0, tf.float32, [1, embedding_size], name='c2')

embedding_Q_ = tf.concat([c1, c2], 0, name='embedding_Q_')
embedding_Q = tf.Variable(tf.truncated_normal(shape=[num_items, embedding_size], 
                                              mean=0.0, stddev=0.01), name='embedding_Q', dtype=tf.float32)

bias = tf.Variable(tf.zeros(num_items), name='bias')
weights = {'out': tf.Variable(tf.random_normal([n_hidden[-1], 1], mean=0, 
                                stddev=np.sqrt(2.0 / (n_hidden[-1] + 1))), name='weights_out')}

biases = {'out': tf.Variable(tf.random_normal([1]), name='biases_out')}
n_hidden_0 = embedding_size

for i in range(len(n_hidden)):
    if i > 0:
        n_hidden_0 = n_hidden[i - 1]
    n_hidden_1 = n_hidden[i]
    weights['h%d' % i] = tf.Variable(tf.random_normal([n_hidden_0, n_hidden_1], mean=0, 
                                    stddev=np.sqrt(2.0 / (n_hidden_0 + n_hidden_1))), name='weights_h%d' % i)
    biases['b%d' % i] = tf.Variable(tf.random_normal([n_hidden_1]), name='biases_b%d' % i)

    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# Creating Inference    
embedding_p = tf.reduce_sum(tf.nn.embedding_lookup(embedding_Q_, user_input), 1)
embedding_q = tf.reduce_sum(tf.nn.embedding_lookup(embedding_Q, item_input), 1)
bias_i = tf.nn.embedding_lookup(bias, item_input)
coeff = tf.pow(num_idx, -tf.constant(alpha, tf.float32, [1]))
embedding_p = coeff * embedding_p

layer1 = tf.multiply(embedding_p, embedding_q)
for i in range(len(n_hidden)):
    layer1 = tf.add(tf.matmul(layer1, weights['h%d' % i]), biases['b%d' % i])
#     if use_batch_norm:
#         layer1 = batch_norm_layer(layer1, train_phase=is_train_phase, scope_bn='bn_%d' % i)
    layer1 = tf.nn.relu(layer1)
out_layer = tf.matmul(layer1, weights['out']) + biases['out']

output = tf.sigmoid(tf.add_n([out_layer, bias_i]))


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# Creating loss
loss = tf.losses.log_loss(labels, output) + \
            lambda_bilinear * tf.reduce_sum(tf.square(embedding_Q)) + \
            gamma_bilinear * tf.reduce_sum(tf.square(embedding_Q_))

for i in range(min(len(n_hidden), len(reg_W))):
    if reg_W[i] > 0:
        loss = loss + reg_W[i] * tf.reduce_sum(tf.square(weights['h%d' % i]))
        

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# Creating optimizer
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate,
                initial_accumulator_value=1e-8).minimize(loss)

        

### Training the Model

In [ ]:
# weight_path = '../datasets/Pretraining/ml-1m/16/alpha0.0.ckpt'
# saver = tf.train.Saver([c1, embedding_Q, bias])

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# # Use pretrained
# saver.restore(sess, weight_path)
# p_c1, p_e_Q, p_b = sess.run([c1, embedding_Q, bias])

# c1 = tf.Variable(p_c1, dtype=tf.float32, trainable=True, name='c1')
# embedding_Q_ = tf.concat([model.c1, model.c2], 0, name='embedding_Q_')
# embedding_Q = tf.Variable(p_e_Q, dtype=tf.float32, trainable=True, name='embedding_Q')
# bias = tf.Variable(p_b, dtype=tf.float32, trainable=True, name='embedding_Q')

# logging.info("using pretrained variables")
# print("using pretrained variables")



# initialize for training batches
batch_begin = time()
batches = shuffle(trainMatrix, trainList, num_negatives)
batch_time = time() - batch_begin

print(batches)
# num_batch = len(batches[1])
# batch_index = list(range(num_batch))



# # # initialize the evaluation feed_dicts
# # testDict = evaluate.init_evaluate_model(model, sess, dataset.testRatings, dataset.testNegatives,
# #                                         dataset.trainList)



# best_hr, best_ndcg = 0, 0
# # train by epoch
# for epoch_count in range(epochs):

#     train_begin = time()
#     for index in batch_index:
#         user_input, num_idx, item_input, labels = [(batches[r])[index] for r in range(4)] 
        
#         # Train a  batch
#         feed_dict = {user_input: user_input, num_idx: num_idx[:, None],
#                      item_input: item_input[:, None],
#                      labels: labels[:, None], is_train_phase: True}
#         sess.run(optimizer, feed_dict)
#     train_time = time() - train_begin

    


#     #################################################################################
# #     if epoch_count % verbose == 0:
# #         train_loss = 0.0
# #         num_batch = len(batches[1])
        
# #         for index in range(num_batch):
# #             user_input, num_idx, item_input, labels = [(batches[r])[index] for r in range(4)]
            
# #             feed_dict = {user_input: user_input, model.num_idx: num_idx[:, None],
# #                          item_input: item_input[:, None],
# #                          labels: labels[:, None], model.is_train_phase: True}
# #             train_loss += sess.run(loss, feed_dict)

# #         (hits, ndcgs, losses) = evaluate.eval(model, sess, dataset.testRatings, dataset.testNegatives, testDict)
# #         hr, ndcg, test_loss = np.array(hits).mean(), np.array(ndcgs).mean(), np.array(losses).mean()
#     #################################################################################

#     batch_begin = time()
#     batches = shuffle(trainMatrix, trainList, batch_size, num_negatives)
#     np.random.shuffle(batch_index)
#     batch_time = time() - batch_begin





    
    

6040
3706


In [ ]:
sess.close()